In [1]:
import rasterio
import numpy as np
import pandas as pd

In [2]:
def read_hwsd_grid(filname):
    with rasterio.open(filname, 'r') as ds:
        # 读取单波段栅格数据（HWSD为单波段）
        grid_data = ds.read(1)  # 读取第一个波段（该数据集只有1个波段）
        return(grid_data)

In [ ]:
# 查找target_mu值的栅格坐标
def find_mu_global_grids(target_mu,grid_data):
    rows, cols = np.where(grid_data == target_mu)
    return list(zip(rows, cols))  # 返回(行号, 列号)列表

In [ ]:
# 计算机经纬度
def rowcol_to_lonlat(row, col):

    lon_res = 0.00833333333333
    lat_res = -0.00833333333333
    west_lon = -179.99583333333334
    north_lat = 89.99583333326137

    # 计算栅格中心点经度（西边界+列号×分辨率）
    lon = west_lon + col * lon_res + lon_res / 2 #经度
    lat = north_lat + row* lat_res + lat_res / 2 #纬度
    
    return(lon,lat)

In [ ]:
# 查找MU_GLOBAL=7006的所有栅格对应的经纬度
def search_lonlat(target_mu_value, grid_data):

    lon_result = []
    lat_result = []
    
    grids = find_mu_global_grids(target_mu_value,grid_data)
    
    for kk in range(len(grids)):
        lon, lat = rowcol_to_lonlat(grids[kk][0],grids[kk][1])
        lon_result.append(lon)
        lat_result.append(lat)
        
    return (lon_result, lat_result)

In [ ]:
# 加载数据
def load_data(filename):
    
    usecols = ['ID', 'MU_GLOBAL', 'T_OC', 'T_PH_H2O', 'T_BULK_DENSITY', 'T_CLAY', 'T_CEC_SOIL']
    renames = ['ID', 'MU_GLOBAL', 'SOC', 'pH', 'BD', 'Clay', 'CEC'] 
    df = pd.read_excel(filename, usecols=usecols, index_col='ID')
    df = df.rename(columns=dict(zip(usecols,renames)))

    return(df)

In [ ]:
if __name__ == "__main__":

    df = load_data('HWSD_DATA.xlsx')
    df = df.dropna()
    result_df = pd.DataFrame()
    grid_data = read_hwsd_grid('hwsd.bil')
    
    for k in df.index[:1000]:
        lon_result, lat_result = search_lonlat(df.loc[k, 'MU_GLOBAL'], grid_data)
        clay = [df.loc[k,'Clay']]*len(lon_result)
        bd = [df.loc[k,'BD']]*len(lon_result)
        soc = [df.loc[k,'SOC']]*len(lon_result)
        ph = [df.loc[k,'pH']]*len(lon_result)
        cec = [df.loc[k,'CEC']]*len(lon_result)
    
        match_df = pd.DataFrame({'lat':lat_result, 'lon':lon_result, 'Clay':clay, 'BD':bd, 'SOC':soc, 'pH':ph, 'CEC':cec})
        
        if result_df.empty:
            result_df = match_df
        else:
            # 按经纬度匹配合并，确保同一经纬度的月份数据对应
            result_df = pd.concat([result_df, match_df], axis=0, ignore_index=True)

        if k%10==0:
            print("正在进行中....")
            
    result_df.to_csv('./soil_data1.csv', index=False)

In [ ]:
result_df.to_csv('./soil_data1.csv', index=False)